In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 0.25
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.linewidth'] = 0.25

import torch, numpy, os
from scipy.io import loadmat
from IPython.display import display
from importlib import reload

Now, load up the training data set for a GAN

In [ ]:
from seeing import fsd
cachedir = 'results/fsd/cache'
true_churches = 'datasets/lsun/church_outdoor_train'
pgan_churches = 'results/imagesample/church/size_10000'

true_churches_tally, pgan_churches_tally = [
    fsd.cached_tally_directory(d, size=10000, cachedir=cachedir, seed=1)
    for d in [true_churches, pgan_churches]]


true_bedrooms = 'datasets/lsun/bedroom_train'
pgan_bedrooms = 'results/imagesample/bedroom/size_10000'

true_bedrooms_tally, pgan_bedrooms_tally = [
    fsd.cached_tally_directory(d, size=10000, cachedir=cachedir, seed=1)
    for d in [true_bedrooms, pgan_bedrooms]]



In [ ]:
from seeing import segmenter

def print_stats(result):
    upp = segmenter.UnifiedParsingSegmenter()
    labelnames, catnames = upp.get_label_and_category_names()
    for label in (-result).sort(0)[1]:
        if label == 0 or labelnames[label][1] == 'material':
            continue
        if result[label] == 0:
            break
        print(label.item(), labelnames[label][0], result[label].item())

# print_stats(gresult)

In [ ]:
def print_diff(tresult, gresult):
    upp = segmenter.UnifiedParsingSegmenter()
    labelnames, catnames = upp.get_label_and_category_names()
    for label in (-tresult).sort(0)[1]:
        if label == 0 or labelnames[label][1] == 'material':
            continue
        if tresult[label] == 0:
            break
        print(label.item(), labelnames[label][0], tresult[label].item(),
             (gresult[label] - tresult[label]).item(),
             ((gresult[label] - tresult[label]).float() / tresult[label]).item())
# print_diff(tresult, gresult)

In [ ]:
import matplotlib.pyplot as plt

def plot_diff(ttally, gtally, title='Objects in Generated vs Training scenes',
              count=30, labelleft=True, dpi=100, legend=False):
    tresult, gresult = [t.mean(0) for t in [ttally, gtally]]
    upp = segmenter.UnifiedParsingSegmenter()
    labelnames, catnames = upp.get_label_and_category_names()
    x = []
    labels = []
    gen_amount = []
    change_frac = []
    true_amount = []
    for label in numpy.argsort(-tresult):
        if label == 0 or labelnames[label][1] == 'material':
            continue
        if tresult[label] == 0:
            break
        x.append(len(x))
        labels.append(labelnames[label][0].split()[0])
        true_amount.append(tresult[label].item())
        gen_amount.append(gresult[label].item())
        change_frac.append((float(gresult[label] - tresult[label])
                            / tresult[label]))
        if len(x) >= count:
            break
    f, (a1, a0) = plt.subplots(2, 1, gridspec_kw = {'height_ratios':[1.2, 2]}, dpi=dpi)
    
    a0.bar(x, change_frac, label='relative delta', color='cornflowerblue')
    a0.set_xticks(x)
    a0.set_xticklabels(labels, rotation='vertical')
    a0.set_ylabel('relative delta\n(gen - train) / train')
    a0.set_ylim([-1, 1.1])
    a0.grid(axis='y', antialiased=False, alpha=0.25)
    if legend:
        a0.legend(loc=2)
    #a0.text(1 if labelleft else 30, 0.9, 'relative delta',
    #       horizontalalignment='left' if labelleft else 'right', verticalalignment='top',
    #       )
    prev_high = None
    for ix, cf in enumerate(change_frac):
        if cf > 1.15:
            if prev_high == (ix - 1):
                offset = 0.1
            else:
                offset = 0.0
                prev_high = ix
            a0.text(ix, 1.15 + offset, '%.1f' % cf, horizontalalignment='center', size=6)
            
    a1.bar(x, true_amount, label='training', color='cornflowerblue')
    a1.plot(x, gen_amount, linewidth=3, color='red', label='generated')
    a1.set_yscale('log')
    a1.set_ylim(1e-2, 50)
    a1.set_yticks([1e-2, 1e-1, 1e+0, 1e+1])

    a1.set_ylabel('mean area\nlog scale')
    # a1.text(30, numpy.max(true_amount), 'mean area in training data, log scale',
    #       horizontalalignment='right', verticalalignment='top',
    #       )
    # a1.set_title(title)
    if legend:
        a1.legend()

    # plt.subplots_adjust(bottom=0.10)
    a1.set_xticks([])
    # Pad margins so that markers don't get clipped by the axes
    # plt.margins(0.2)
    f.tight_layout()
    plt.show()



In [ ]:
from seeing import frechet_distance

print(frechet_distance.sample_frechet_distance(true_churches_tally * 100, pgan_churches_tally * 100))


In [ ]:
plt.style.use('dark_background')
plot_diff(true_churches_tally*100, pgan_churches_tally*100, title='', dpi=500)

In [ ]:
plot_diff(true_bedrooms_tally*100, pgan_bedrooms_tally*100, title='', dpi=500)

In [ ]:
for mname in arch_tally_100.keys():
    print(mname, frechet_distance.sample_frechet_distance(bedroom_result_100[0], arch_tally_100[mname]))


In [ ]:
numpy.argmax(arch_tally_100['minibatch-stddev'][2])

In [ ]:
ablated_bedroom_stats = tally_ablated_model(SETTINGS.bedroom)

In [ ]:
if 'wgangp' in arch_tally_100:
    plot_diff(bedroom_result_100[0][:10000], arch_tally_100['wgangp'][:10000],
          title='Objects in wgangp Generated vs Training Bedrooms', dpi=500)

In [ ]:
if 'proggan' in arch_tally_100:
    plot_diff(bedroom_result_100[0][:10000], arch_tally_100['proggan'][:10000],
          title='Objects in with progressive training Generated vs Training Bedrooms', dpi=500)

In [ ]:
if 'minibatch-stddev' in arch_tally_100:
    plot_diff(bedroom_result_100[0][:10000], arch_tally_100['minibatch-stddev'][:10000],
          title='Objects in with progressive training Generated vs Training Bedrooms', dpi=500)

In [ ]:
if 'stylegan' in arch_tally_100:
    plot_diff(bedroom_result_100[0][:10000], arch_tally_100['stylegan'][:10000],
          title='Objects in with stylegan Generated vs Training Bedrooms', dpi=500, legend=True)

In [ ]:
plot_diff(*bedroom_result_100,
          title='Objects in Generated vs Training Bedrooms', dpi=100)

In [ ]:
plot_diff(bedroom_result_100[0][:10000], inverted_tally,
          title='Objects in inverted images Generated vs Training Bedrooms', dpi=100)

In [ ]:
plot_diff(bedroom_result_100[0][:10000], net_inverted_tally,
          title='Objects in inverted images Generated vs Training Bedrooms', dpi=100)

In [ ]:
arch_tally_100['stylegan'].shape

In [ ]:
print(frechet_distance.sample_frechet_distance(bedroom_result_100[0], bedroom_result_100[1]))


In [ ]:
bedroom_result[0].max()

In [ ]:
bedroom_result[1].shape

In [ ]:
plot_diff(*bedroom_result_100,
          title='Objects in Generated vs Training Bedrooms', dpi=500, legend=True)

In [ ]:
plot_diff(*church_result_100,
         title='Objects in Generated vs Training Churches', dpi=500, legend=True)

In [ ]:
def images_maximizing_labels(setting):
    dataset = load_dataset(setting, 'train')
    loader = DataLoader(dataset,
                        # sampler=FixedRandomSubsetSampler(dataset, end=1000),
                        sampler=FixedSubsetSampler(range(1000)),
                        batch_size=10, pin_memory=True)
    upp = segmenter.UnifiedParsingSegmenter()
    best_index = torch.zeros(NUM_OBJECTS, dtype=torch.long)
    best = torch.zeros(NUM_OBJECTS, dtype=torch.long).cuda()
    with torch.no_grad():
        batch_index = 0
        for [batch] in progress(loader):
            seg_result = upp.segment_batch(batch.cuda())
            for i in range(len(batch)):
                result = seg_result[i,0].view(-1).bincount(minlength=NUM_OBJECTS)
                records = (result > best).nonzero()
                for label in records:
                    best_index[label] = batch_index + i
                    best[label] = result[label]
            batch_index += len(batch)
    return best_index

In [ ]:
bedroom_examples = images_maximizing_labels(SETTINGS.bedroom)
church_examples = images_maximizing_labels(SETTINGS.church)

In [ ]:
[i for i, (n, c) in enumerate(labelnames) if n == 'car']

In [ ]:
church_examples[13]

In [ ]:
dataset = parallelfolder.ParallelImageFolders(
            ['/data/vision/torralba/scratch2/davidbau/newdissect/imagesample/compare_arch/size_10000/minibatch-stddev/baseline/'],
            transform=transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(256),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                ]))

In [ ]:
classnum = 2
modelname = 'minibatch-stddev'

def show_top_seg_examples(modelname, classnum):
    upp = segmenter.UnifiedParsingSegmenter()
    print(upp.get_label_and_category_names()[0][classnum][0])
    dataset = load_arch_dataset(modelname)
    top_10_images = [dataset.images[i][0]
                     for i in numpy.argsort(arch_tally_100[modelname][:,classnum])[-10:][::-1]]
    for filename in top_10_images:
        im = Image.open(filename)
        display(im.resize((256, 256)))
        img = ((transforms.functional.to_tensor(im.resize([256,256])) - 0.5) * 2)[None,:,:,:]
        seg_result = upp.raw_seg_prediction(img.cuda())
        obj_seg = seg_result[0]['object']
        obj_labels = obj_seg.max(1)[1].cpu().numpy()
        # Recolor some labels
        # obj_labels[obj_labels == 5] = 12
        # obj_labels[obj_labels == 4] = 5
        sv = segviz.segment_visualization(recolored(obj_labels), (256, 256))
        display(Image.fromarray(sv))
        obj2 = obj_labels.copy()
        obj2[obj2 != classnum] = 0
        sv2 = segviz.segment_visualization(recolored(obj2), (256, 256))
        display(Image.fromarray(sv2))

def recolored(labels):
    print(labels.shape)
    bldg = (labels == 5)
    tree = (labels == 4)
    chair = (labels == 12)
    labels_rc = labels.copy()
    labels_rc[bldg] = 12
    labels_rc[tree] = 5
    labels_rc[chair] = 4
    return labels_rc

show_top_seg_examples('stylegan', 2)

In [ ]:
re

In [ ]:
dataset.images[8371]

In [ ]:
from PIL import Image
from netdissect import segmenter
from netdissect import segviz
from torchvision import transforms
import numpy

# filename = '/data/vision/torralba/scratch2/davidbau/encoder/results/bedroom/optimize_residuals/cases/opt_n_136/images/136_a_3000.png'
filename = '/data/vision/torralba/scratch2/davidbau/encoder/results/bedroom/optimize_residuals/cases/opt_n_136/images/136_target.png'
# filename = '/data/vision/torralba/scratch2/davidbau/encoder/results/bedroom/optimize_residuals/summary_s/316_target.png'
# filename = '/data/vision/torralba/scratch2/davidbau/encoder/results/bedroom/optimize_residuals/summary_s/316_a_3000.png'
# filename = '/data/vision/torralba/scratch2/davidbau/newdissect/imagesample/compare_arch/size_10000/minibatch-stddev/baseline/image_1647.png'
# filename = '/data/vision/torralba/scratch2/davidbau/encoder/results/church/optimize_residuals/summary_n/883_target.png'
# filename = '/data/vision/torralba/scratch2/davidbau/encoder/results/diningroom/optimize_residuals/cases/opt_n_81/images/81_target.png'
# filename = '/data/vision/torralba/scratch2/davidbau/encoder/results/diningroom/optimize_residuals/cases/opt_n_81/images/81_a_3000.png'
# filename = '/data/vision/torralba/scratch2/davidbau/encoder/results/diningroom/optimize_residuals/cases/opt_n_124/images/124_target.png'
# filename = '/data/vision/torralba/scratch2/davidbau/encoder/results/diningroom/optimize_residuals/cases/opt_n_124/images/124_a_3000.png'
# filename = '/data/vision/torralba/scratch2/davidbau/encoder/results/livingroom/optimize_residuals/cases/opt_n_5/images/5_a_3000.png'
# filename = '/data/vision/torralba/scratch2/davidbau/encoder/results/livingroom/optimize_residuals/cases/opt_n_5/images/5_target.png'
# filename = '/data/vision/torralba/scratch2/davidbau/encoder/results/livingroom/optimize_residuals/cases/opt_n_34/images/34_a_3000.png'
# filename = '/data/vision/torralba/scratch2/davidbau/encoder/results/livingroom/optimize_residuals/cases/opt_n_34/images/34_target.png'
# filename = '/data/vision/torralba/scratch2/davidbau/encoder/results/kitchen/optimize_residuals/cases/opt_i_8/images/8_a_3000.png'
# filename = '/data/vision/torralba/scratch2/davidbau/encoder/results/kitchen/optimize_residuals/cases/opt_i_8/images/8_target.png'
#filename = '/data/vision/torralba/scratch2/davidbau/encoder/results/kitchen/optimize_residuals/cases/opt_i_24/images/24_a_3000.png'
filename = '/data/vision/torralba/scratch2/davidbau/encoder/results/kitchen/optimize_residuals/cases/opt_i_24/images/24_target.png'

im = Image.open(filename)
display(im.resize((256, 256)))
img = ((transforms.functional.to_tensor(im.resize([256,256])) - 0.5) * 2)[None,:,:,:]

with torch.no_grad():
    upp = segmenter.UnifiedParsingSegmenter()
    seg_result = upp.raw_seg_prediction(img.cuda())
    obj_seg = seg_result[0]['object']

obj_labels = recolored(obj_seg.max(1)[1].cpu().numpy())
# Recolor some labels
# obj_labels[obj_labels == 5] = 12
# obj_labels[obj_labels == 4] = 5
sv = segviz.segment_visualization(obj_labels, (256, 256))
display(Image.fromarray(sv))

for label in numpy.argsort(numpy.bincount(obj_labels.flatten()))[-15:][::-1]:
    obj2 = obj_labels.copy()
    obj2[obj2 != label] = 0
    sv2 = segviz.segment_visualization(obj2, (256, 256))
    display(Image.fromarray(sv2))
    print(upp.get_label_and_category_names()[0]['label'][0])


In [ ]:
upp.get_label_and_category_names()

In [ ]:
numpy.argmax(arch_tally_100['minibatch-stddev'][:,2])

In [ ]:
numpy.bincount(obj_labels.flatten())